<a href="https://colab.research.google.com/github/gdollp/mlb/blob/main/submit2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook uses lightGBM to make predictions.

We use the following features
* playerId
* position
* teamId(rosters)
* status(rosters)
* playerBoxScores

and the date 20200401~20200431 as the validation data.

But I think there is room for improvement.  
If you have better ways, I would appreciate it if you could comment on it.

このnotebookではlightGBMを使って予測します。

特徴量は以下のものを使用しています。
* playerId
* position
* teamId(rosters)
* status(rosters)
* playerBoxScores

20200401~20200431を日時をvalidation dataとしていますが、一考の余地がありそうです。  
もし良さそうな方法があればコメントしていただけると幸いです。

https://www.kaggle.com/columbia2131/mlb-lightgbm-starter-dataset-code-en-ja

## About Dataset

Train.csv is stored as a csv file with each column as follows.  

train.csvを以下のようにして各カラムをcsvファイルとして保管しています。

In [ ]:
%%capture
"""
!pip install pandarallel 

import gc

import numpy as np
import pandas as pd
from pathlib import Path

from pandarallel import pandarallel
pandarallel.initialize()

BASE_DIR = Path('../input/mlb-player-digital-engagement-forecasting')
train = pd.read_csv(BASE_DIR / 'train.csv')

null = np.nan
true = True
false = False

for col in train.columns:

    if col == 'date': continue

    _index = train[col].notnull()
    train.loc[_index, col] = train.loc[_index, col].parallel_apply(lambda x: eval(x))

    outputs = []
    for index, date, record in train.loc[_index, ['date', col]].itertuples():
        _df = pd.DataFrame(record)
        _df['index'] = index
        _df['date'] = date
        outputs.append(_df)

    outputs = pd.concat(outputs).reset_index(drop=True)

    outputs.to_csv(f'{col}_train.csv', index=False)
    outputs.to_pickle(f'{col}_train.pkl')

    del outputs
    del train[col]
    gc.collect()
"""

In [ ]:
!pip install optuna

## Training

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error
from datetime import timedelta
from functools import reduce
from tqdm import tqdm
import optuna.integration.lightgbm as lgbm
#import lightgbm as lgbm
#import mlb
import pickle
from datetime import datetime as dt
import copy
import gc
from sklearn.decomposition import PCA
import math
from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
BASE_DIR = Path('/content/drive/MyDrive/mlb/input')
TRAIN_DIR = Path('/content/drive/MyDrive/mlb/input/archive')

In [ ]:
players = pd.read_csv(BASE_DIR / 'players.csv')

rosters = pd.read_pickle(TRAIN_DIR / 'rosters_train.pkl')
targets = pd.read_pickle(TRAIN_DIR / 'nextDayPlayerEngagement_train.pkl')
scores1 = pd.read_pickle(TRAIN_DIR / 'playerBoxScores_train.pkl')
scores = scores1.groupby(['playerId', 'date']).sum().reset_index()
twitter = pd.read_pickle("/content/drive/MyDrive/mlb/input/archive/playerTwitterFollowers_train.pkl")
games = pd.read_pickle(TRAIN_DIR / 'games_train.pkl')
events = pd.read_pickle(TRAIN_DIR / 'events_train.pkl')
standings = pd.read_pickle(TRAIN_DIR / 'standings_train.pkl')
teamtwitter = pd.read_pickle(TRAIN_DIR / 'teamTwitterFollowers_train.pkl')
transaction = pd.read_pickle(TRAIN_DIR / 'teamTwitterFollowers_train.pkl')
awards = pd.read_csv(BASE_DIR / 'awards.csv')
seasons = pd.read_csv(BASE_DIR / 'seasons.csv')
teams = pd.read_csv(BASE_DIR / 'teams.csv')
player_target_stats = pd.read_csv("/content/drive/MyDrive/mlb/input/player_target_stats.csv")

example_test = pd.read_csv("/content/drive/MyDrive/mlb/input/example_test.csv")

In [ ]:
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

test = unpack_json(example_test["rosters"].iloc[0])
test["playerId"]
del example_test
gc.collect()

1195

In [ ]:
awards2 = awards.groupby("playerId").count()
awards2 = awards2.reset_index()

In [ ]:
teamtwitter["teamnumberOfFollowers"] = teamtwitter["numberOfFollowers"]
teamtwi = teamtwitter.groupby("teamId").mean()["teamnumberOfFollowers"].reset_index()

In [ ]:
players

,playerId,playerName,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
0,665482,Gilberto Celestino,1999-02-13,2021-06-02,Santo Domingo,NaN,Dominican Republic,72,170,8,Outfielder,False
1,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True
2,661269,Vladimir Gutierrez,1995-09-18,2021-05-28,Havana,NaN,Cuba,73,190,1,Pitcher,True
3,669212,Eli Morgan,1996-05-13,2021-05-28,Rancho Palos Verdes,CA,USA,70,190,1,Pitcher,True
4,666201,Alek Manoah,1998-01-09,2021-05-27,Homestead,FL,USA,78,260,1,Pitcher,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2056,672695,Geraldo Perdomo,1999-10-22,NaN,Santo Domingo,NaN,Dominican Republic,74,203,6,Shortstop,True
2057,672911,Jesus Cruz,1995-04-15,NaN,Salinas de Hidalgo,San Luis Potosi,Mexico,73,230,1,Pitcher,False
2058,676103,Damon Jones,1994-09-30,NaN,Twin Falls,ID,USA,77,233,1,Pitcher,True
2059,676755,Isaac Mattson,1995-07-14,NaN,Erie,PA,USA,74,205,1,Pitcher,True


In [ ]:
games = pd.read_pickle(TRAIN_DIR / 'games_train.pkl')
#games.index[games["detailedGameState"] == "Postponed"].shape
#games.drop(games.loc[games['detailedGameState']=='Postponed'].index, inplace=True)
#games["detailedGameState"].unique()
#games[games.duplicated(subset=["gamePk"], keep=False)]
#games['detailedGameState']=='Postponed']だとスコアはキロクされていない

In [ ]:
targets["engagementMetricsDate"] = targets["engagementMetricsDate"].str.replace('-', '')
yesterday_targets = targets.drop('date', axis=1)
yesterday_targets = yesterday_targets.rename(columns={'engagementMetricsDate':'date', 'target1': 'yest_target1','target2': 'yest_target2','target3': 'yest_target3','target4': 'yest_target4'})
yesterday_targets["date"] = yesterday_targets["date"].astype(int)
yesterday_targets["date"]

0          20180102
1          20180102
2          20180102
3          20180102
4          20180102
             ...   
2506171    20210501
2506172    20210501
2506173    20210501
2506174    20210501
2506175    20210501
Name: date, Length: 2506176, dtype: int64

In [ ]:
targets_cols = ['playerId', 'target1', 'target2', 'target3', 'target4', 'date']
players_cols = ['playerId', 'primaryPositionName',"playerName"]
rosters_cols = ['playerId', 'teamId', 'status', 'date']
yesterday_targets_cols = ["date","playerId","yest_target1","yest_target2","yest_target3","yest_target4"]
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances', 'date',"gamePk"]
games_cols = ["gamePk","homeId","dayNight","seriesDescription","gamesInSeries","homeWinner","awayWinner","homeScore","awayScore","gameType",
              "gameDate"]
playertwitter_cols = ["playerId","numberOfFollowers"]
awards_cols = ["playerId","awardName"]
standings_cols = ["date","teamId","divisionRank","divisionLeader","wildCardLeader","leagueRank","divisionId","gameDate"]
teamtwitter_cols = ["teamId","teamnumberOfFollowers"]
targets["engagementMetricsDate"] = targets["engagementMetricsDate"].str.replace('-', '')
yesterday_targets = targets.drop('date', axis=1)
yesterday_targets = yesterday_targets.rename(columns={'engagementMetricsDate':'date', 'target1': 'yest_target1','target2': 'yest_target2','target3': 'yest_target3','target4': 'yest_target4'})
yesterday_targets["date"] = yesterday_targets["date"].astype(int)
yesterday_targets["date"]

0          20180102
1          20180102
2          20180102
3          20180102
4          20180102
             ...   
2506171    20210501
2506172    20210501
2506173    20210501
2506174    20210501
2506175    20210501
Name: date, Length: 2506176, dtype: int64

In [ ]:
feature_cols = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored','homeRuns',
       'strikeOuts', 'baseOnBalls',  'hits', 'hitByPitch',
       'atBats', 'stolenBases', 
       'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacFlies',
       'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching','winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching',  'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored',
       'sacFliesPitching', 'saves', 'holds',
       'assists', 'putOuts', 'errors', 'chances','target1_mean',
 'target1_median',
 'target1_std',
 'target1_min',
 'target1_max',
 'target1_prob',
 'target2_mean',
 'target2_median',
 'target2_std',
 'target2_min',
 'target2_max',
 'target2_prob',
 'target3_mean',
 'target3_median',
 'target3_std',
 'target3_min',
 'target3_max',
 'target3_prob',
 'target4_mean',
 'target4_median',
 'target4_std',
 'target4_min',
 'target4_max',
 'target4_prob',"divisionId","teamnumberOfFollowers","preseasonhits"]
feature_cols2 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored',# 'doubles', 'triples', 'homeRuns',
       'baseOnBalls', 'hits',
      # 'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'plateAppearances', 'totalBases', 'rbi',
      # 'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'winsPitching',
       'lossesPitching',# 'flyOutsPitching', 'airOutsPitching',
       'runsPitching', 
       'strikeOutsPitching',
       #'hitsPitching',
       'hitByPitchPitching',  'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched',
        'battersFaced', 
        'balks', 'pickoffsPitching',
        'inheritedRunners',
        
       #'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'putOuts','chances','target1_mean',
 'target1_median',
 'target1_std',
 'target1_min',
 'target1_max',
 'target1_prob',
 'target2_mean',
 'target2_median',
 'target2_std',
 'target2_min',
 'target2_max',
 'target2_prob',
 'target3_mean',
 'target3_median',
 'target3_std',
 'target3_min',
 'target3_max',
 'target3_prob',
 'target4_mean',
 'target4_median',
 'target4_std',
 'target4_min',
 'target4_max',
 'target4_prob',
    'target1',"divisionId","teamnumberOfFollowers","preseasonhits"]
feature_cols3 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', #'gamesPlayedBatting', 'flyOuts',
        'homeRuns',
      # 'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
     # 'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
        'totalBases', 'rbi',
      # 'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
        'gamesStartedPitching',
      # 'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 
      # 'groundOutsPitching', 'runsPitching', 'doublesPitching',
      # 'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       #'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       #'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'inningsPitched', 
        'battersFaced', 'pitchesThrown', 
      # 'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
      # 'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       #'inheritedRunnersScored', 'catchersInterferencePitching',
       #'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
      # 'assists', 'putOuts', 'errors', 'chances',
      'target1_mean',
 'target1_median',
 'target1_std',
 'target1_min',
 'target1_max',
 'target1_prob',
 'target2_mean',
 'target2_median',
 'target2_std',
 'target2_min',
 'target2_max',
 'target2_prob',
 'target3_mean',
 'target3_median',
 'target3_std',
 'target3_min',
 'target3_max',
 'target3_prob',
 'target4_mean',
 'target4_median',
 'target4_std',
 'target4_min',
 'target4_max',
 'target4_prob',
    "divisionId","teamnumberOfFollowers","target2","preseasonhits"]
feature_cols4 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
        'runsScored','triples','groundIntoDoublePlay',
       'strikeOuts', 'baseOnBalls','hits', 
       'atBats', 'caughtStealing',
       'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies',
        'gamesPlayedPitching',
        'winsPitching',
       'airOutsPitching',
       'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
        'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 
        'inningsPitched', 'saveOpportunities',
       'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes','wildPitches', 
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 
       'sacBuntsPitching', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances','target1_mean',
 'target1_median',
 'target1_std',
 'target1_min',
 'target1_max',
 'target1_prob',
 'target2_mean',
 'target2_median',
 'target2_std',
 'target2_min',
 'target2_max',
 'target2_prob',
 'target3_mean',
 'target3_median',
 'target3_std',
 'target3_min',
 'target3_max',
 'target3_prob',
 'target4_mean',
 'target4_median',
 'target4_std',
 'target4_min',
 'target4_max',
 'target4_prob',
 'target1', "divisionId","target2","target3","teamnumberOfFollowers","preseasonhits"]

In [ ]:
twitter["strdate"] = twitter["date"].astype(str)
twitter["year_months"] = twitter["strdate"].str[0:6].astype(int)


In [ ]:
targets['hasTwitterAccount'] = targets.playerId.isin(twitter.playerId)

In [ ]:
train = targets[targets_cols].merge(players[players_cols], on=['playerId'], how='left')
train = train.merge(rosters[rosters_cols], on=['playerId', 'date'], how='left')
train = train.merge(scores[scores_cols], on=['playerId', 'date'], how='left')
train = train.merge(games[games_cols], on=["gamePk"], how="left")
train = train.merge(standings[standings_cols], on=['date',"teamId"], how='left')
train = train.merge(awards2[awards_cols], on=['playerId'], how='left')
train = train.merge(player_target_stats, how='inner', left_on=["playerId"],right_on=["playerId"])
train["strdate"] = train["date"].astype(str)
train["year_months"] = train["strdate"].str[0:6].astype(int)
#train = train.merge(twitter[playertwitter_cols], on=['playerId'], how='left')
train = train.merge(teamtwi[teamtwitter_cols], on=['teamId'], how='left')



In [ ]:
targets['hasTwitterAccount'] = targets.playerId.isin(twitter.playerId)

In [ ]:
test["playerId"]

0       405395
1       408234
2       444482
3       445276
4       446334
         ...  
1264    676477
1265    676755
1266    676831
1267    676845
1268    676886
Name: playerId, Length: 1269, dtype: int64

In [ ]:
train = train[train["playerId"].isin(test["playerId"].tolist())]


In [ ]:
train[['target1_mean',
 'target1_median',
 'target1_std',
 'target1_min',
 'target1_max',
 'target1_prob',
 'target2_mean',
 'target2_median',
 'target2_std',
 'target2_min',
 'target2_max',
 'target2_prob',
 'target3_mean',
 'target3_median',
 'target3_std',
 'target3_min',
 'target3_max',
 'target3_prob',
 'target4_mean',
 'target4_median',
 'target4_std',
 'target4_min',
 'target4_max',
 'target4_prob',
    'target1',"divisionId","target2","target3"]].describe()

,target1_mean,target1_median,target1_std,target1_min,target1_max,target1_prob,target2_mean,target2_median,target2_std,target2_min,target2_max,target2_prob,target3_mean,target3_median,target3_std,target3_min,target3_max,target3_prob,target4_mean,target4_median,target4_std,target4_min,target4_max,target4_prob,target1,divisionId,target2,target3
count,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,1.425738e+06,431035.000000,1.425738e+06,1.425738e+06
mean,1.412069e+00,3.666530e-01,2.596503e+00,8.686041e-03,1.166149e+01,1.954660e+00,3.598050e+00,2.126944e+00,4.203402e+00,5.287085e-01,1.908462e+01,4.265614e+00,9.861317e-01,1.324842e-01,2.424857e+00,1.291285e-02,1.159615e+01,6.186417e-01,2.355667e+00,1.605815e+00,2.481981e+00,3.121415e-01,1.145428e+01,2.596091e+00,8.546450e-01,202.569190,3.296722e+00,1.014046e+00
std,2.968291e+00,1.351763e+00,4.648054e+00,3.511933e-02,2.081765e+01,7.745529e+00,4.494150e+00,3.418236e+00,5.057487e+00,9.294232e-01,2.236597e+01,1.002152e+01,2.313200e+00,6.944196e-01,4.553159e+00,3.688311e-02,2.081837e+01,3.697182e+00,4.055712e+00,3.243416e+00,3.988353e+00,6.498519e-01,1.775528e+01,7.197724e+00,5.169614e+00,1.710673,7.403711e+00,6.228724e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.169842e-02,2.746197e-02,2.463763e-02,0.000000e+00,1.098479e-01,9.348415e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,200.000000,0.000000e+00,0.000000e+00
25%,7.606870e-03,8.005113e-04,1.957108e-02,0.000000e+00,1.068279e-01,7.275680e-04,8.289821e-01,3.341743e-01,9.325237e-01,9.294914e-02,4.237536e+00,2.966081e-01,8.837747e-03,3.280409e-03,9.032549e-03,0.000000e+00,4.533348e-02,2.287021e-03,4.301866e-01,2.622378e-01,4.335073e-01,4.637816e-02,1.937359e+00,1.884002e-01,0.000000e+00,201.000000,1.411765e-01,0.000000e+00
50%,2.453845e-01,7.016000e-03,6.412187e-01,9.107007e-04,2.947525e+00,7.194967e-03,2.148966e+00,1.109506e+00,2.302353e+00,2.696926e-01,1.076173e+01,1.402635e+00,2.036182e-01,1.260653e-02,6.181471e-01,1.814676e-03,3.184691e+00,1.143511e-02,1.023956e+00,6.381039e-01,1.094529e+00,1.200061e-01,5.137918e+00,6.229110e-01,3.387852e-03,203.000000,9.576815e-01,4.475961e-03
75%,1.441729e+00,1.078931e-01,2.842854e+00,4.949999e-03,1.255925e+01,2.685799e-01,5.030897e+00,2.612921e+00,4.822524e+00,6.019906e-01,2.417816e+01,4.099971e+00,9.091083e-01,5.593215e-02,2.528056e+00,9.390976e-03,1.207621e+01,6.202865e-02,2.538233e+00,1.549681e+00,2.655361e+00,2.920946e-01,1.252833e+01,1.953503e+00,5.868221e-02,204.000000,3.309058e+00,4.991568e-02
max,2.861945e+01,2.251564e+01,3.398812e+01,7.040277e-01,1.000000e+02,1.000000e+02,4.969193e+01,4.431772e+01,3.288215e+01,1.550033e+01,1.000000e+02,1.000000e+02,3.218307e+01,1.590317e+01,3.533198e+01,6.190328e-01,1.000000e+02,6.304345e+01,3.933404e+01,3.842898e+01,3.410180e+01,8.647749e+00,1.000000e+02,1.000000e+02,1.000000e+02,205.000000,1.000000e+02,1.000000e+02


In [ ]:
train[["target1","target2","target3","target4"]].describe()
train[train["target1"]==train["target1"].max()]
train[train["playerId"]==519317]

,playerId,target1,target2,target3,target4,date,primaryPositionName,playerName,teamId,status,battingOrder,gamesPlayedBatting,flyOuts,groundOuts,runsScored,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,atBats,caughtStealing,stolenBases,groundIntoDoublePlay,groundIntoTriplePlay,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,catchersInterference,pickoffs,gamesPlayedPitching,gamesStartedPitching,completeGamesPitching,shutoutsPitching,...,homeWinner,awayWinner,homeScore,awayScore,gameType,gameDate_x,divisionRank,divisionLeader,wildCardLeader,leagueRank,divisionId,gameDate_y,awardName,target1_mean,target1_median,target1_std,target1_min,target1_max,target1_prob,target2_mean,target2_median,target2_std,target2_min,target2_max,target2_prob,target3_mean,target3_median,target3_std,target3_min,target3_max,target3_prob,target4_mean,target4_median,target4_std,target4_min,target4_max,target4_prob,strdate,year_months,teamnumberOfFollowers
2435,519317,0.974327,56.177043,13.693746,64.166667,20180101,Designated Hitter,Giancarlo Stanton,147.0,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,16.57155,6.296399,24.090019,0.087622,100.0,15.112738,17.019935,15.442759,8.480908,4.489628,33.868643,12.036823,5.245218,0.40027,11.07997,0.077679,39.703651,2.993143,13.301593,12.0,8.016774,2.059741,34.008584,6.248675,20180101,201801,3399726.125
2436,519317,0.743145,31.715912,11.397756,47.248120,20180102,Designated Hitter,Giancarlo Stanton,147.0,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,16.57155,6.296399,24.090019,0.087622,100.0,15.112738,17.019935,15.442759,8.480908,4.489628,33.868643,12.036823,5.245218,0.40027,11.07997,0.077679,39.703651,2.993143,13.301593,12.0,8.016774,2.059741,34.008584,6.248675,20180102,201801,3399726.125
2437,519317,0.741518,30.998168,25.975685,79.879622,20180103,Designated Hitter,Giancarlo Stanton,147.0,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,16.57155,6.296399,24.090019,0.087622,100.0,15.112738,17.019935,15.442759,8.480908,4.489628,33.868643,12.036823,5.245218,0.40027,11.07997,0.077679,39.703651,2.993143,13.301593,12.0,8.016774,2.059741,34.008584,6.248675,20180103,201801,3399726.125
2438,519317,0.434217,40.828745,6.552563,86.740331,20180104,Designated Hitter,Giancarlo Stanton,147.0,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,16.57155,6.296399,24.090019,0.087622,100.0,15.112738,17.019935,15.442759,8.480908,4.489628,33.868643,12.036823,5.245218,0.40027,11.07997,0.077679,39.703651,2.993143,13.301593,12.0,8.016774,2.059741,34.008584,6.248675,20180104,201801,3399726.125
2439,519317,1.700575,20.981956,26.427778,34.202094,20180105,Designated Hitter,Giancarlo Stanton,147.0,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,16.57155,6.296399,24.090019,0.087622,100.0,15.112738,17.019935,15.442759,8.480908,4.489628,33.868643,12.036823,5.245218,0.40027,11.07997,0.077679,39.703651,2.993143,13.301593,12.0,8.016774,2.059741,34.008584,6.248675,20180105,201801,3399726.125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3652,519317,14.208138,16.863095,6.

In [ ]:
train["stryear"] = train["date"].astype(str)
train["year"] = train["stryear"].str[0:4].astype(int)

In [ ]:
#make feature
train["strdate"] = train["date"].astype(str)
train["month"] = train["strdate"].str[5].astype(int)
train["month"]
train["stryear"] = train["date"].astype(str)
train["year"] = train["stryear"].str[0:4].astype(int)
train["nullc"] = train.isnull().sum(axis=1)
train["ongame"] = np.where(train["gamePk"].isnull() == 1,0,1)
train["ongame"].unique()

array([0, 1])

In [ ]:
del awards
del scores
del rosters
del standings

In [ ]:
# label encoding
player2num = {c: i for i, c in enumerate(train['playerId'].unique())}
position2num = {c: i for i, c in enumerate(train['primaryPositionName'].unique())}
teamid2num = {c: i for i, c in enumerate(train['teamId'].unique())}
status2num = {c: i for i, c in enumerate(train['status'].unique())}
daynight2num = {c: i for i, c in enumerate(train['dayNight'].unique())}
seriesDescription2num = {c: i for i, c in enumerate(train['seriesDescription'].unique())}
gameType2num = {c: i for i, c in enumerate(train['gameType'].unique())}
#bitrhCountry2num = {c: i for i, c in enumerate(train["birthCountry"].unique())}
train['label_playerId'] = train['playerId'].map(player2num)
train['label_primaryPositionName'] = train['primaryPositionName'].map(position2num)
train['label_teamId'] = train['teamId'].map(teamid2num)
train['label_status'] = train['status'].map(status2num)
train["label_daynight"] = train['dayNight'].map(daynight2num)
train["label_seriesDescription"] = train["seriesDescription"].map(seriesDescription2num)
#train["birthCountry"] = train["birthCountry"].map(bitrhCountry2num)

from sklearn.compose import TransformedTargetRegressor


In [ ]:
#train.to_csv("/content/drive/MyDrive/mlb/script/output/train.csv")

In [ ]:
playerlist = train["playerId"].unique()

In [ ]:
train["playerId"]

0          628317
1          628317
2          628317
3          628317
4          628317
            ...  
2508779    656775
2508780    656775
2508781    656775
2508782    656775
2508783    656775
Name: playerId, Length: 1425738, dtype: int64

In [ ]:

train_2017post = train[train["date"]<=20180328]
train_2018reg = train[(train["date"]>=20180329)&(train["date"]<=20181001)]
train_2018post = train[(train["date"]>=20181002)&(train["date"]<=20190319)]
train_2019reg = train[(train["date"]>=20190320)&(train["date"]<=20190929)]
train_2019post = train[(train["date"]>=20190930)&(train["date"]<=20200722)]
train_2020reg = train[(train["date"]>=20200723)&(train["date"]<=20200927)]
train_2020post = train[(train["date"]>=20200928)&(train["date"]<=20210331)]
train_2021reg = train[(train["date"]>=20210401)&(train["date"]<=20211003)]


In [ ]:
seasons

,seasonId,seasonStartDate,seasonEndDate,preSeasonStartDate,preSeasonEndDate,regularSeasonStartDate,regularSeasonEndDate,lastDate1stHalf,allStarDate,firstDate2ndHalf,postSeasonStartDate,postSeasonEndDate
0,2017,2017-04-02,2017-11-01,2017-02-22,2017-04-01,2017-04-02,2017-10-01,2017-07-09,2017-07-11,2017-07-14,2017-10-03,2017-11-01
1,2018,2018-03-29,2018-10-28,2018-02-21,2018-03-27,2018-03-29,2018-10-01,2018-07-15,2018-07-17,2018-07-19,2018-10-02,2018-10-28
2,2019,2019-03-20,2019-10-30,2019-02-21,2019-03-26,2019-03-20,2019-09-29,2019-07-07,2019-07-09,2019-07-11,2019-10-01,2019-10-30
3,2020,2020-07-23,2020-10-28,2020-02-21,2020-07-22,2020-07-23,2020-09-27,2020-08-25,NaN,2020-08-26,2020-09-29,2020-10-28
4,2021,2021-02-28,2021-10-31,2021-02-28,2021-03-30,2021-04-01,2021-10-03,2021-07-11,2021-07-13,2021-07-15,2021-10-04,2021-10-31


In [ ]:
#make feature
hitscol = ["playerId","preseasonhits"]
hits_2018 = train_2018reg.groupby("playerId").sum()["hits"].reset_index()
hits_2018["preseasonhits"] = hits_2018["hits"]
train_2018post = train_2018post.merge(hits_2018[hitscol], on=['playerId'], how='left')
train_2019reg = train_2019reg.merge(hits_2018[hitscol], on=['playerId'], how='left')

hits2019col = ["playerId","hits2019"]
hits_2019 = train_2019reg.groupby("playerId").sum()["hits"].reset_index()
hits_2019["preseasonhits"] = hits_2019["hits"]
train_2019post = train_2019post.merge(hits_2019[hitscol], on=['playerId'], how='left')
train_2020reg = train_2020reg.merge(hits_2019[hitscol], on=['playerId'], how='left')

hits2020col = ["playerId","hits2020"]
hits_2020 = train_2020reg.groupby("playerId").sum()["hits"].reset_index()
hits_2020["preseasonhits"] = hits_2020["hits"]
train_2020post = train_2020post.merge(hits_2020[hitscol], on=['playerId'], how='left')
train_2021reg = train_2021reg.merge(hits_2020[hitscol], on=['playerId'], how='left')
del hits_2018,hits_2019,hits_2020
gc.collect()

gamescol = ["playerId","preseasonpt"]
game2018 = train_2018reg.groupby("playerId").sum()["ongame"].reset_index()
game2018["preseasonpt"] = game2018["ongame"]
train_2018post = train_2018post.merge(game2018[gamescol], on=['playerId'], how='left')
train_2019reg = train_2019reg.merge(game2018[gamescol], on=['playerId'], how='left')

game2019 = train_2019reg.groupby("playerId").sum()["ongame"].reset_index()
game2019["preseasonpt"] = game2019["ongame"]
train_2019post = train_2019post.merge(game2019[gamescol], on=['playerId'], how='left')
train_2020reg = train_2020reg.merge(game2019[gamescol], on=['playerId'], how='left')

game2020 = train_2020reg.groupby("playerId").sum()["ongame"].reset_index()
game2020["preseasonpt"] = game2020["ongame"]
train_2020post = train_2020post.merge(game2020[gamescol], on=['playerId'], how='left')
train_2021reg = train_2021reg.merge(game2020[gamescol], on=['playerId'], how='left')

del game2018,game2019,game2020
gc.collect()

0

In [ ]:
train = pd.concat([train_2017post,train_2018reg,train_2018post,train_2019reg,train_2019post,train_2020reg,train_2020post,train_2021reg])
train["preseasonhits"]
del train_2017post,train_2018post,train_2019post, train_2020post, train_2018reg, train_2019reg, train_2020reg

In [ ]:
train["hitpergame"] = train["preseasonhits"] / train["preseasonpt"]

In [ ]:

train_y = train[['target1', 'target2', 'target3', 'target4']]


_index = (train['date'] < 20210401)
gc.collect()


x_train = train.loc[_index].reset_index(drop=True)
y_train = train_y.loc[_index].reset_index(drop=True)
x_valid = train.loc[~_index].reset_index(drop=True)
y_valid = train_y.loc[~_index].reset_index(drop=True)



gc.collect()

0

In [ ]:
train

,playerId,target1,target2,target3,target4,date,primaryPositionName,playerName,teamId,status,battingOrder,gamesPlayedBatting,flyOuts,groundOuts,runsScored,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,atBats,caughtStealing,stolenBases,groundIntoDoublePlay,groundIntoTriplePlay,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,catchersInterference,pickoffs,gamesPlayedPitching,gamesStartedPitching,completeGamesPitching,shutoutsPitching,...,target1_median,target1_std,target1_min,target1_max,target1_prob,target2_mean,target2_median,target2_std,target2_min,target2_max,target2_prob,target3_mean,target3_median,target3_std,target3_min,target3_max,target3_prob,target4_mean,target4_median,target4_std,target4_min,target4_max,target4_prob,strdate,year_months,teamnumberOfFollowers,stryear,year,month,nullc,ongame,label_playerId,label_primaryPositionName,label_teamId,label_status,label_daynight,label_seriesDescription,preseasonhits,preseasonpt,hitpergame
0,628317,0.011167,4.474708,0.005168,5.735294,20180101,Pitcher,Kenta Maeda,119.0,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.024572,1.372738,0.003102,7.193825,0.038621,7.595162,3.207558,11.575222,0.793096,49.982843,5.240148,0.369429,0.130267,0.733202,0.018696,3.236853,0.244629,2.997316,2.331033,2.187388,0.597849,9.648069,3.094059,20180101,201801,2137878.1,20180101,2018,1,90,0,0,0,0,0,0,0,NaN,NaN,NaN
1,628317,0.006356,2.521008,0.001648,2.766917,20180102,Pitcher,Kenta Maeda,119.0,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.024572,1.372738,0.003102,7.193825,0.038621,7.595162,3.207558,11.575222,0.793096,49.982843,5.240148,0.369429,0.130267,0.733202,0.018696,3.236853,0.244629,2.997316,2.331033,2.187388,0.597849,9.648069,3.094059,20180102,201801,2137878.1,20180102,2018,1,90,0,0,0,0,0,0,0,NaN,NaN,NaN
2,628317,0.003556,2.380952,0.003624,3.439381,20180103,Pitcher,Kenta Maeda,119.0,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.024572,1.372738,0.003102,7.193825,0.038621,7.595162,3.207558,11.575222,0.793096,49.982843,5.240148,0.369429,0.130267,0.733202,0.018696,3.236853,0.244629,2.997316,2.331033,2.187388,0.597849,9.648069,3.094059,20180103,201801,2137878.1,20180103,2018,1,90,0,0,0,0,0,0,0,NaN,NaN,NaN
3,628317,0.003363,3.332367,0.012294,4.849601,20180104,Pitcher,Kenta Maeda,119.0,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.024572,1.372738,0.003102,7.193825,0.038621,7.595162,3.207558,11.575222,0.793096,49.982843,5.240148,0.369429,0.130267,0.733202,0.018696,3.236853,0.244629,2.997316,2.331033,2.187388,0.597849,9.648069,3.094059,20180104,201801,2137878.1,20180104,2018,1,90,0,0,0,0,0,0,0,NaN,NaN,NaN
4,628317,0.023976,1.440761,0.001501,2.371288,20180105,Pitcher,Kenta Maeda,119.0,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.024572,1.372738,0.003102,7.193825,0.038621,7.595162,3.207558,11.575222,0.793096,49.982843,5.240148,0.369429,0.130267,0.733202,0.018696,3.236853,0.244629,2.997316,2.331033,2.187388,0.597849,9.648069,3.094059,20180105,201801,2137878.1,20180105,2018,1,90,0,0,0,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35135,656775,16.920553,13.339693,5.035756,4.136498,20210426,Outfielder,Cedric Mullins,110.0,Active,100.0,1.0,1.0,0.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,10.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [ ]:
def fit_lgbm(x_train, y_train, x_valid, y_valid, params: dict=None, verbose=100):
    oof_pred = np.zeros(len(y_valid), dtype=np.float32)
    #model = lgbm.LGBMRegressor(**params)
    best_params, tuning_history = {}, []
    
    lgb_train = lgbm.Dataset(x_train, y_train)
    lgb_eval = lgbm.Dataset(x_valid, y_valid, reference=lgb_train)
    model = lgbm.train(params, 
        lgb_train, valid_sets=lgb_eval,  
        early_stopping_rounds=100,
        num_boost_round=500,
        verbose_eval=100,
        #verbose=verbose,               
        )
    oof_pred = model.predict(x_valid)
    score = mean_absolute_error(oof_pred, y_valid)
    print('mae:', score)
    return oof_pred, model, score



# training lightgbm
params1 = {'objective':'mae','reg_alpha': 0.14947461820098767, 'reg_lambda': 0.10185644384043743, 'n_estimators': 3633, 'learning_rate': 0.08046301304430488, 'num_leaves': 674, 'feature_fraction': 0.9101240539122566, 'bagging_fraction': 0.9884451442950513, 'bagging_freq': 8, 'min_child_samples': 51}

params2 = {
 'objective':'mae',
 'reg_alpha': 0.1,
 'reg_lambda': 0.1, 
 'n_estimators': 5000,
 'learning_rate': 0.1,
 'random_state': 42,
 "num_leaves": 22
}

params4 = {'objective':'mae','reg_alpha': 0.016468100279441976, 'reg_lambda': 0.09128335764019105, 'n_estimators': 9868, 'learning_rate': 0.10528150510326864, 'num_leaves': 157, 'feature_fraction': 0.5419185713426886, 'bagging_fraction': 0.2637405128936662, 'bagging_freq': 19, 'min_child_samples': 71}

params = {
 'objective':'mae',
 'metric': 'mae',
 'reg_alpha': 0.1,
 'reg_lambda': 0.1, 
 'n_estimators': 10000,
 'learning_rate': 0.1,
 'random_state': 42,
 "num_leaves": 100
}


oof1, model1, score1 = fit_lgbm(
    x_train[feature_cols], y_train['target1'],
    x_valid[feature_cols], y_valid['target1'],
    params
)

file = '/content/drive/MyDrive/mlb/script/output/model1.pkl'
pickle.dump(model1, open(file, 'wb'))

oof2, model2, score2 = fit_lgbm(
    x_train[feature_cols2], y_train['target2'],
    x_valid[feature_cols2], y_valid['target2'],
    params
)

file = '/content/drive/MyDrive/mlb/script/output/model2.pkl'
pickle.dump(model2, open(file, 'wb'))

oof3, model3, score3 = fit_lgbm(
    x_train[feature_cols3], y_train['target3'],
    x_valid[feature_cols3], y_valid['target3'],
    params
)

file = '/content/drive/MyDrive/mlb/script/output/model3.pkl'
pickle.dump(model3, open(file, 'wb'))


oof4, model4, score4 = fit_lgbm(
    x_train[feature_cols4], y_train['target4'],
    x_valid[feature_cols4], y_valid['target4'],
    params
)


file = '/content/drive/MyDrive/mlb/script/output/model4.pkl'
pickle.dump(model4, open(file, 'wb'))
score = (score1+score2+score3+score4) / 4
print(f'score: {score}')


[I 2021-07-08 08:11:00,160] A new study created in memory with name: no-name-c2279d68-1073-4e43-8ea6-7ceec513980b

feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning:

Found `n_estimators` in params. Will use it instead of argument



Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 1.2198
[200]	valid_0's l1: 1.21861
[300]	valid_0's l1: 1.21852



feature_fraction, val_score: 1.218513:  14%|#4        | 1/7 [00:23<02:18, 23.03s/it][I 2021-07-08 08:11:23,219] Trial 0 finished with value: 1.218513294084598 and parameters: {'feature_fraction': 0.8999999999999999}. Best is trial 0 with value: 1.218513294084598.

feature_fraction, val_score: 1.218513:  14%|#4        | 1/7 [00:23<02:18, 23.03s/it]

Early stopping, best iteration is:
[244]	valid_0's l1: 1.21851
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 1.22254
[200]	valid_0's l1: 1.22164



feature_fraction, val_score: 1.218513:  29%|##8       | 2/7 [00:43<01:51, 22.28s/it][I 2021-07-08 08:11:43,753] Trial 1 finished with value: 1.221643439062955 and parameters: {'feature_fraction': 0.7}. Best is trial 0 with value: 1.218513294084598.

feature_fraction, val_score: 1.218513:  29%|##8       | 2/7 [00:43<01:51, 22.28s/it]

...


min_data_in_leaf, val_score: 1.354559:  80%|########  | 4/5 [08:23<01:50, 110.93s/it][I 2021-07-08 11:13:58,629] Trial 63 finished with value: 1.369602500036572 and parameters: {'min_child_samples': 10}. Best is trial 61 with value: 1.3545591855673302.

min_data_in_leaf, val_score: 1.354559:  80%|########  | 4/5 [08:23<01:50, 110.93s/it]

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 1.41372
[200]	valid_0's l1: 1.3951
[300]	valid_0's l1: 1.38873
[400]	valid_0's l1: 1.38416
[500]	valid_0's l1: 1.38176
[600]	valid_0's l1: 1.37619
[700]	valid_0's l1: 1.37579



min_data_in_leaf, val_score: 1.354559: 100%|##########| 5/5 [09:11<00:00, 110.22s/it]

Early stopping, best iteration is:
[611]	valid_0's l1: 1.37566


mae: 1.3545591854990213
score: 1.4179844013094776


In [ ]:
players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances']
standings_cols = ["teamId","divisionRank","divisionLeader","wildCardLeader","leagueRank","divisionId","gameDate"]
null = np.nan
true = True
false = False

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
from tqdm import tqdm
import gc
from functools import reduce
from sklearn.model_selection import StratifiedKFold

ROOT_DIR = "../input/mlb-player-digital-engagement-forecasting"

#=======================#
def flatten(df, col):
    du = (df.pivot(index="playerId", columns="EvalDate", 
               values=col).add_prefix(f"{col}_").
      rename_axis(None, axis=1).reset_index())
    return du
#============================#
def reducer(left, right):
    return left.merge(right, on="playerId")
#========================

TGTCOLS = ["target1","target2","target3","target4"]
def train_lag(df, lag=1):
    dp = df[["playerId","EvalDate"]+TGTCOLS].copy()
    dp["EvalDate"]  =dp["EvalDate"] + timedelta(days=lag) 
    df = df.merge(dp, on=["playerId", "EvalDate"], suffixes=["",f"_{lag}"], how="left")
    return df
#=================================
def test_lag(sub):
    sub["playerId"] = sub["date_playerId"].apply(lambda s: int(  s.split("_")[1]  ) )
    assert sub.date.nunique() == 1
    dte = sub["date"].unique()[0]
    
    eval_dt = pd.to_datetime(dte, format="%Y%m%d")
    dtes = [eval_dt + timedelta(days=-k) for k in LAGS]
    mp_dtes = {eval_dt + timedelta(days=-k):k for k in LAGS}
    
    sl = LAST.loc[LAST.EvalDate.between(dtes[-1], dtes[0]), ["EvalDate","playerId"]+TGTCOLS].copy()
    sl["EvalDate"] = sl["EvalDate"].map(mp_dtes)
    du = [flatten(sl, col) for col in TGTCOLS]
    du = reduce(reducer, du)
    return du, eval_dt
    #
#===============

tr = pd.read_csv("../input/mlb-data/target.csv")
print(tr.shape)
gc.collect()

tr["EvalDate"] = pd.to_datetime(tr["EvalDate"])
tr["EvalDate"] = tr["EvalDate"] + timedelta(days=-1)
tr["EvalYear"] = tr["EvalDate"].dt.year

MED_DF = tr.groupby(["playerId","EvalYear"])[TGTCOLS].median().reset_index()
MEDCOLS = ["tgt1_med","tgt2_med", "tgt3_med", "tgt4_med"]
MED_DF.columns = ["playerId","EvalYear"] + MEDCOLS

LAGS = list(range(1,21))
FECOLS = [f"{col}_{lag}" for lag in reversed(LAGS) for col in TGTCOLS]

for lag in tqdm(LAGS):
    tr = train_lag(tr, lag=lag)
    gc.collect()
#===========
tr = tr.sort_values(by=["playerId", "EvalDate"])
print(tr.shape)
tr = tr.dropna()
print(tr.shape)
tr = tr.merge(MED_DF, on=["playerId","EvalYear"])
gc.collect()

X = tr[FECOLS+MEDCOLS].values
y = tr[TGTCOLS].values
cl = tr["playerId"].values

NFOLDS = 6
skf = StratifiedKFold(n_splits=NFOLDS)
folds = skf.split(X, cl)
folds = list(folds)

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

tf.random.set_seed(777)

def make_model(n_in):
    inp = L.Input(name="inputs", shape=(n_in,))
    x = L.Dense(50, activation="relu", name="d1")(inp)
    x = L.Dense(50, activation="relu", name="d2")(x)
    preds = L.Dense(4, activation="linear", name="preds")(x)
    
    model = M.Model(inp, preds, name="ANN")
    model.compile(loss="mean_absolute_error", optimizer="adam")
    return model

net = make_model(X.shape[1])
print(net.summary())

oof = np.zeros(y.shape)
nets = []
for idx in range(NFOLDS):
    print("FOLD:", idx)
    tr_idx, val_idx = folds[idx]
    ckpt = ModelCheckpoint(f"w{idx}.h5", monitor='val_loss', verbose=1, save_best_only=True,mode='min')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=3, min_lr=0.0005)
    es = EarlyStopping(monitor='val_loss', patience=6)
    reg = make_model(X.shape[1])
    reg.fit(X[tr_idx], y[tr_idx], epochs=10, batch_size=35_000, validation_data=(X[val_idx], y[val_idx]),
            verbose=1, callbacks=[ckpt, reduce_lr, es])
    reg.load_weights(f"w{idx}.h5")
    oof[val_idx] = reg.predict(X[val_idx], batch_size=50_000, verbose=1)
    nets.append(reg)
    gc.collect()
    #
#

mae = mean_absolute_error(y, oof)
mse = mean_squared_error(y, oof, squared=False)
print("mae:", mae)
print("mse:", mse)

# Historical information to use in prediction time
bound_dt = pd.to_datetime("2021-01-01")
LAST = tr.loc[tr.EvalDate>bound_dt].copy()

LAST_MED_DF = MED_DF.loc[MED_DF.EvalYear==2021].copy()
LAST_MED_DF.drop("EvalYear", axis=1, inplace=True)
del tr

#"""
import mlb
FE = []; SUB = [];

FileNotFoundError: ignored

## Inference

In [ ]:
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

In [ ]:
"""players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances']

null = np.nan
true = True
false = False"""

In [ ]:
"""players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances',"gamePk"]

null = np.nan
true = True
false = False

env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

for (test_df, sample_prediction_df) in iter_test: # make predictions here
    
    sub = copy.deepcopy(sample_prediction_df.reset_index())
    sample_prediction_df = copy.deepcopy(sample_prediction_df.reset_index(drop=True))
    
    
    # creat dataset
    sample_prediction_df['playerId'] = sample_prediction_df['date_playerId']\
                                        .map(lambda x: int(x.split('_')[1]))
    # Dealing with missing values
    if test_df['rosters'].iloc[0] == test_df['rosters'].iloc[0]:
        test_rosters = pd.DataFrame(eval(test_df['rosters'].iloc[0]))
    else:
        test_rosters = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in rosters.columns:
            if col == 'playerId': continue
            test_rosters[col] = np.nan
            
    if test_df['playerBoxScores'].iloc[0] == test_df['playerBoxScores'].iloc[0]:
        test_scores = pd.DataFrame(eval(test_df['playerBoxScores'].iloc[0]))
    else:
        test_scores = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in scores.columns:
            if col == 'playerId': continue
            test_scores[col] = np.nan
    
    test_games = unpack_json(test_df["games"].iloc[0])
    test_standings = unpack_json(test_df["standings"].iloc[0])
    
            
            
    

    



        
    test_scores = test_scores.groupby('playerId').sum().reset_index()
    test = sample_prediction_df[['playerId']].copy()
    test = test.merge(players[players_cols], on='playerId', how='left')
    test = test.merge(test_rosters[rosters_cols], on='playerId', how='left')
    test = test.merge(test_scores[scores_cols], on='playerId', how='left')
    test = test.merge(test_games[games_cols], on="gamePk", how="left")
    test = test.merge(awards2[awards_cols], on=['playerId'], how='left')
    test = test.merge(player_target_stats, how='inner', left_on=["playerId"],right_on=["playerId"])
    test = test.merge(test_standings[standings_cols], on=["teamId"], how='left')
    
    #add feature
    test["ongame"] = np.where(test["gamePk"].isnull() == 1,0,1)
    test["ishome"] = np.where(test["teamId"]==test["homeId"],2,test["ongame"])
    test["winorlose"] = np.where(test["teamId"]==test["homeId"],test["homeWinner"],test["awayWinner"])
    test["winorlose"] = test["winorlose"].fillna(2.0).astype(int)
    test["score"] = np.where(test["teamId"]==test["homeId"],test["homeScore"],test["awayScore"])
    test["divisionRank"] = test["divisionRank"].fillna(7.0).astype(int)
    test["divisionLeader"] = test["divisionLeader"].fillna(-1.0).astype(int)
    test["wildCardLeader"] = np.where(test["wildCardLeader"]=="True",1,0)
    
    
    
    
    #label encoding
    test['label_playerId'] = test['playerId'].map(player2num)
    test['label_primaryPositionName'] = test['primaryPositionName'].map(position2num)
    test['label_teamId'] = test['teamId'].map(teamid2num)
    test['label_status'] = test['status'].map(status2num)
    test["label_daynight"] = test['dayNight'].map(daynight2num)
    test["label_seriesDescription"] = test["seriesDescription"].map(seriesDescription2num)
    test["gameType"] = test["gameType"].map(gameType2num)
    display(test)
    
    
    test_X1 = test[feature_cols1]
    test_X2 = test[feature_cols2]
    test_X3 = test[feature_cols3]
    test_X4 = test[feature_cols4]
    
    # predict
    pred1 = model1.predict(test_X1)
    pred2 = model2.predict(test_X2)
    pred3 = model3.predict(test_X3)
    pred4 = model4.predict(test_X4)
    
    # merge submission
    sample_prediction_df['target1'] = np.clip(pred1, 0, 100)
    sample_prediction_df['target2'] = np.clip(pred2, 0, 100)
    sample_prediction_df['target3'] = np.clip(pred3, 0, 100)
    sample_prediction_df['target4'] = np.clip(pred4, 0, 100)
    sample_prediction_df = sample_prediction_df.fillna(0.)
    del sample_prediction_df['playerId']
    
    # TF summit
    # Features computation at Evaluation Date
    sub_fe, eval_dt = test_lag(sub)
    sub_fe = sub_fe.merge(LAST_MED_DF, on="playerId", how="left")
    sub_fe = sub_fe.fillna(0.)
    
    _preds = 0.
    for reg in nets:
        _preds += reg.predict(sub_fe[FECOLS + MEDCOLS]) / NFOLDS
    sub_fe[TGTCOLS] = np.clip(_preds, 0, 100)
    sub.drop(["date"]+TGTCOLS, axis=1, inplace=True)
    sub = sub.merge(sub_fe[["playerId"]+TGTCOLS], on="playerId", how="left")
    sub.drop("playerId", axis=1, inplace=True)
    sub = sub.fillna(0.)
    # Blending
    blend = pd.concat(
        [sub[['date_playerId']],
        (0.1*sub.drop('date_playerId', axis=1) + 0.9*sample_prediction_df.drop('date_playerId', axis=1))],
        axis=1
    )
    env.predict(blend)
    # Update Available information
    sub_fe["EvalDate"] = eval_dt
    #sub_fe.drop(MEDCOLS, axis=1, inplace=True)
    LAST = LAST.append(sub_fe)
    LAST = LAST.drop_duplicates(subset=["EvalDate","playerId"], keep="last")"""

In [ ]:
display(test_df['games'])

In [ ]:
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

In [ ]:
unpack_json(test_df["games"].iloc[0])

In [ ]:
pd.concat(
    [sub[['date_playerId']],
    (sub.drop('date_playerId', axis=1) + sample_prediction_df.drop('date_playerId', axis=1)) / 2],
    axis=1
)

In [ ]:
sample_prediction_df